# Analysis of Missing reconstruction pulses

In [ ]:
# Do standard imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits, ascii
from astropy.table import Table
import os
import auxiliary 

In [ ]:
nsims = 100
fluxDir = f"{os.getcwd()}/flux0.50mcrab/"
exposure = 4331
sampling_rate=130210 #Hz
auxiliary.verbose = 0

In [ ]:
import ast
missing_distances = list()
for i in range(nsims):
    isim = i + 1
    csv_file = f"{fluxDir}/sim_{isim}/00_info_nofilt_infoc_sim{isim}_missing.csv"
    missing_table = pd.read_csv(csv_file, converters={"Non-reconstructed photons": ast.literal_eval,"Bad-reconstructed photons": ast.literal_eval})
    #missing_table["Bad-reconstructed photons"] = missing_table["Bad-reconstructed photons"].apply(np.array)
    #missing_table["Non-reconstructed photons"] = missing_table["Non-reconstructed photons"].apply(np.array)
   
    #read table row by row:
    # Frist column is an integer value that represents the pixel id
    # Second column is a list of integers that represents the PH_ID of the missing photons
    # Third column is a list of integers that represents the PH_ID of the photons that are bad reconstructed    
    for i, row in missing_table.iterrows():
        ipixel = row["Pixel"]
        missing_phs_id = row["Non-reconstructed photons"]
        bad_recons_phs_id = row["Bad-reconstructed photons"]
        auxiliary.vprint(f"sim {isim}, pixel {ipixel}, missing photons {missing_phs_id}, bad reconstructed photons {bad_recons_phs_id}")
        # identify piximpact file for pixel
        piximpact_file = f"{fluxDir}/sim_{isim}/crab_flux0.50_Emin2_Emax10_exp{exposure}_RA0.0_Dec0.0_nofilt_infoc_pixel{ipixel}_piximpact.fits"
        # create a dictionary: keys are the ipixels; values are lists of missing distances
        # read TIME and PH_ID columns of piximpact FITS file

        with fits.open(piximpact_file) as hdul:
            piximpact_data = hdul[1].data
            time = piximpact_data["TIME"].copy()
            ph_id = piximpact_data["PH_ID"].copy()
        # foreach missing photon, find the minimum TIME distance to the bad reconstructed photons (find its 'partner')
        for imissing in missing_phs_id:
            missing_time = time[ph_id == imissing][0] 
            min_time_diff_samples = float("inf")   
            # find the bad reconstructed photon that is closest in time to the missing photon
            min_bad = None
            for ibad in bad_recons_phs_id:
                bad_time = time[ph_id == ibad][0]
                time_diff_samples = np.abs(missing_time - bad_time)*sampling_rate
                if time_diff_samples < min_time_diff_samples:
                    min_time_diff_samples = time_diff_samples
                    min_bad = ibad
            if min_bad is None:
                print(f"sim {isim}, pixel {ipixel}: no bad ph for missing ph {imissing}")
                raise ValueError("No bad reconstructed photon found for missing photon")
            if min_time_diff_samples > 100:
                print(f"sim {isim}, pixel {ipixel}: missing ph {imissing} and bad ph {min_bad} are separated by {min_time_diff_samples:.2f} samples")
                raise ValueError("Time difference between missing and bad reconstructed photons is too large")
            # append the minimum time difference to the list of missing distances
            missing_distances.append(min_time_diff_samples)
            auxiliary.vprint(f"sim {isim}, pixel {ipixel}, missing ph {imissing}, bad ph {min_bad}, min time diff {min_time_diff_samples:.2f}")


In [ ]:
print(np.min(missing_distances), np.max(missing_distances))

In [ ]:
# plot histogram of missing distances in samples
plt.hist(missing_distances, bins=10)
plt.xlabel("Time difference (samples)")
plt.ylabel("Frequency")
plt.title("Histogram of time differences between missing and bad-reconstructed photons")
